# Converting Scraped Data into IDPCentral

Alasdair J G Gray ([ORCID:0000-0002-5711-4872](http://orcid.org/0000-0002-5711-4872))

_Heriot-Watt University, Edinburgh, UK_

## Introduction

IDPCentral is the idea of having a central registry of proteins that are known to be disordered.

We aim to populate the content of the registry with Bioschemas markup that has been scraped using the BMUSE tool.

This notebook goes through the steps of converting the scraped content into the IDPCentral data model.

### To Dos

- ~~Return provenance data to IDPcentral: straightforward add properties from graph~~
- Consider changing base namespace for IDPKG graph to either one you own or Wikidata
  - Is it valid for us to hang our properties off a UniProt ID
  - If we are using UniProt IDs we need to be consistent in our usage
- Add metadata properties and statements
  - Do as a separate query
- Retrieve UniProt label for IDPcentral using SPARQLWrapper to make external call and add properties
- Do we want to connect to Wikidata IDs?
- ~~IDPcentral is not getting updated with entries from mobidb~~
- MobiDB data getting mangled by BMUSE (currently testing with manually fixed file)

## IDPCentral Data Model

The IDPCental data model reuses ideas from [Wikidata](https://www.mediawiki.org/wiki/Wikibase/DataModel) whereby every statement loaded contains a provenance link as to where it was acquired.

- [ ] Document IDPCentral Model

## Data Sources

The following databases have been scraped to populate IDPCentral
- [DisProt](https://www.disprot.org/)
- [MobiDb](https://mobidb.bio.unipd.it/)
- [Protein Ensemble Database](https://proteinensemble.org/)

## Conversion using RDFlib

This is an attempt to achieve the same functionality without using a triplestore.

Load in the RDFLib library.

In [ ]:
from rdflib import ConjunctiveGraph, Graph

Template library used to template queries.

In [ ]:
from string import Template

Import functions to list files in directory

In [ ]:
from glob import glob

Prepare query to extract UniProt and DisProt IRIs.

In [ ]:
idQuery = """
PREFIX schema: <https://schema.org/>
SELECT ?proteinIRI ?uniprot
WHERE {
    GRAPH ?g {
        ?proteinIRI a schema:Protein ;
            schema:sameAs ?uniprot .
        FILTER regex(str(?uniprot), "^(https://www|http://purl).uniprot.org/uniprot/")
    }
}
"""

Templated query for creating the direct properties for the entity, i.e. the data.

In [ ]:
createEntityQuery = Template("""
#PREFIX bsr: <https://bioschemas.org/reference/>
PREFIX bs: <https://bioschemas.org/entity/>
PREFIX pav: <http://purl.org/pav/>
PREFIX schema: <https://schema.org/>
CONSTRUCT {
    bs:${bsAccession} a schema:Protein ;
        schema:identifier ?identifier ;
        schema:name ?name ;
        schema:hasSequenceAnnotation ?annotation ;
        schema:taxonomicRange ?taxonomicRange ;
        schema:hasRepresentation ?representation ;
        schema:sameAs ?sameAs , <${proteinIRI}>.
#    bsr:${proteinIRI} pav:retrievedFrom ?source ;
#        pav:retrievedOn ?date .
}
WHERE {
    GRAPH ?g {
# Bioschemas Minimal Properties
        <${proteinIRI}> a schema:Protein ;
            schema:identifier ?identifier ;
            schema:name ?name ;
# Bioschemas Recommended properties
            schema:hasSequenceAnnotation ?annotation ;
            schema:taxonomicRange ?taxonomicRange ;
# Bioschemas Optional properties
            schema:sameAs ?sameAs .
        OPTIONAL {<${proteinIRI}> schema:hasRepresentation ?representation }
        ?g pav:retrievedFrom ?source ;
            pav:retrievedOn ?date .
    }
}
""")

Templated query for creating the links to the provenance for each statement in the KG.

In [ ]:
createEntityProvenanceQuery = Template("""
PREFIX bs: <https://bioschemas.org/entity/>
PREFIX bsp: <https://bioschemas.org/ns/p/>
PREFIX bss: <https://bioschemas.org/ns/s/>
PREFIX bsr: <https://bioschemas.org/reference/>
PREFIX pav: <http://purl.org/pav/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <https://schema.org/>
CONSTRUCT {
#    bs:${bsAccession} a schema:Protein ;
#        schema:identifier ?identifier ;
#        schema:name ?name ;
#        schema:hasSequenceAnnotation ?annotation ;
#        schema:taxonomicRange ?taxonomicRange ;
#        schema:hasRepresentation ?representation ;
#        schema:sameAs ?sameAs , <${proteinIRI}>.
    bs:${bsAccession} bsp:type [
        prov:wasDerivedFrom bsr:${bsAccession} ;
        bss:type schema:Protein
    ] .
    bsr:${bsAccession} pav:retrievedFrom ?source ;
        pav:retrievedOn ?date .
}
WHERE {
    GRAPH ?g {
# Bioschemas Minimal Properties
        <${proteinIRI}> a schema:Protein .#;
#            schema:identifier ?identifier ;
#            schema:name ?name ;
# Bioschemas Recommended properties
#            schema:hasSequenceAnnotation ?annotation ;
#            schema:taxonomicRange ?taxonomicRange ;
# Bioschemas Optional properties
#            schema:sameAs ?sameAs .
#        OPTIONAL {<${proteinIRI}> schema:hasRepresentation ?representation }
        ?g pav:retrievedFrom ?source ;
            pav:retrievedOn ?date .
    }
}
""")

Method for creating the KG entity and its metadata using the templated queries above.

In [ ]:
def createKGEntity(g, protein, uniprot):
    kgEntity = Graph()
    # Extract UniProt accession to use as an identifier in the Bioschemas namespace
    accession = uniprot[uniprot.rindex('/')+1:]
    query = createEntityQuery.substitute(proteinIRI=protein,bsAccession=accession)
    kgEntity += g.query(query)
    query = createEntityProvenanceQuery.substitute(proteinIRI=protein,bsAccession=accession)
    print(query)
    kgEntity += g.query(query)
    return kgEntity

Function to extract just the triples that IDPCentral are using in their UI

In [ ]:
idpQuery = """
PREFIX idp: <https://example.com/ipd/>
PREFIX pav: <http://purl.org/pav/>
PREFIX schema: <https://schema.org/>
CONSTRUCT {
    ?entry_url idp:name ?entry_name ;
        idp:identifier ?entry_id ;
        idp:sameAs ?uniprot_acc ;
        idp:sequence_range [
            idp:sequence_id ?sequenceID ;
            idp:start ?start ;
            idp:end ?end ;
            idp:range_annotation ?range_annotation
        ] ;
        idp:resource_name ?source ;
        idp:last_update ?date.
}
WHERE {
    GRAPH ?g {
        ?entry_url a schema:Protein ;
            schema:name ?entry_name ;
            schema:identifier ?entry_id ;
            schema:hasSequenceAnnotation ?sequenceID ;
            schema:sameAs ?uniprot_acc .
        FILTER regex(str(?uniprot_acc), "^(https://www|http://purl).uniprot.org/uniprot/")
        ?sequenceID schema:sequenceLocation ?sequenceLocation ;
                  schema:additionalProperty/schema:value/schema:name ?range_annotation .
        ?sequenceLocation schema:rangeStart ?start ;
            schema:rangeEnd ?end.
        ?g pav:retrievedFrom ?source ;
            pav:retrievedOn ?date .
    }
}
"""
def idpExtraction(g):
    return g.query(idpQuery)

Function to run over all files in a specified directory

In [ ]:
def processDataFiles(idpKG, idpModel, directoryLocation):
    processed = 0
    for file in glob(directoryLocation + "*.nq"):
        print("\tProcessing file: %s" % file)
        g = ConjunctiveGraph()
        g.parse(file, format="nquads")
        print("\tSource has %s statements." % len(g))
        # Extract statements for IDPCentral
        idpModel += idpExtraction(g)
        print("\tIDPcentral has %s statements." % len(idpModel))
        # Extract DisProt and UniProt IRIs
        results = g.query(idQuery)
        print("\tID query result has %s statements." % len(results))
        # Convert to IDPCentral model
        for result in results:
            print("\tProtein: %s\n\tUniProt: %s" % (result['proteinIRI'], result['uniprot']))
            resGraph = createKGEntity(g, result['proteinIRI'], result['uniprot'])
            print("\tconvert query has %s statements." % len(resGraph))
            idpKG += resGraph
            print("\tIDPKG has %s statements." % len(idpKG))
        processed += 1
    return processed

Function to output data files for a graph

In [ ]:
def outputFiles(graph, label):
    # print(graph.serialize(format='nt'))
    print("%s has %s statements." % (label, len(graph)))
    graph.serialize(label+'.nt', format='nt')
    graph.serialize(label+'.jsonld', format='json-ld')
    print('Successfully written all triples to %s.nt' % label)

Read in each nq data file in turn

Process each file and convert into IDPCentral model

In [ ]:
idpKG = Graph()
idpModel = Graph()
totalProcessed = 0

print("Processing DisProt...")
numberOfFiles = processDataFiles(idpKG, idpModel, "../scraped-data/disprot/")
print("Processed %d files" % numberOfFiles)
totalProcessed += numberOfFiles

print("Processing MobiDB...")
numberOfFiles = processDataFiles(idpKG, idpModel, "../scraped-data/mobidb/")
print("Processed %d files" % numberOfFiles)
totalProcessed += numberOfFiles

outputFiles(idpModel, "IDPCentral")
outputFiles(idpKG, "IDPKG")
print('Processed %d files' % totalProcessed)

Expecting:
- 5 files to have been procesed
  - 2 from mobidb
  - 3 from DisProt
- IDPKG should have 25 statements
- IDPCentral should have 58 statements